In [ ]:
import os
import math
import numpy as np
import pandas as pd

from sklearn.metrics import roc_curve, auc
from scipy import stats

from src.utils import parse_filename, normalize

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model

import matplotlib as mpl
from matplotlib import gridspec
import matplotlib.pyplot as plt
import seaborn as sns
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{libertine}'
mpl.rc('font', family='serif')

In [ ]:
data_name = "100_100_100_0.2_0.05_0.03_0.5_0.5_d.npy"
out_name = "100_100_100_0.2_0.05_0.03_0.5_0.5_o.npy"
data = np.load(os.path.join(os.getcwd(), "data", "synth", data_name))
is_outlier = np.load(os.path.join(os.getcwd(), "data", "synth", out_name))
params = parse_filename(data_name)
params

In [ ]:
num_devices = params["num_devices"]
dims = params["dims"]
num_data = params["n"]
subspace_frac = ["subspace_frac"]
frac_outlying_devices = params["frac_outlying_devices"]
frac_outlying_data = params["frac_outlying_data"]

T_start = 1
T = 9
n = int(num_data/(T_start+T))